# Clustering - Grupo 03

## **Authors**

- César López Mantecón - 100472092
- Manuel Gómez-Plana Rodríguez - 100472310



## **Repositorio**

Esta práctica se ha llevado a cabo en este [repositorio de github](https://github.com/ManuGPR/AA_P2_G03.git)

## **Introducción**

En este cuaderno se resolverá un problema de clustering usando un dataset de estrellas.

# Carga de dataset

## Carga del dataset por drive

In [14]:
# Para ejecutar el cuaderno en drive
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
stars = pd.read_csv("/content/drive/MyDrive/stars_data.csv") # acceder al csv en drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Carga del dataset en local

In [ ]:
import pandas as pd
stars = pd.read_csv("starts_data.csv")

# 1. Descripción del dataset
Como podemos ver en la salida del código, contamos con un dataset de 240 instancias y 6 características. De estas 6 características, 4 son numéricas y 2 son categóricas.

In [22]:
stars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Temperature     240 non-null    int64  
 1   L               240 non-null    float64
 2   R               240 non-null    float64
 3   A_M             240 non-null    float64
 4   Color           240 non-null    object 
 5   Spectral_Class  240 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 11.4+ KB


# 2. Programación de K-Means
En este apartado, se cubre una implementación propia del algoritmo K-Means, el cuál compararemos luego con el algoritmo de `scikit-learn`.

In [30]:
from random import random
from math import sqrt
import numpy

class MyK_Means:
  def __init__(self, k: int, data: pd.DataFrame):
    self.k = k
    self.data = data

    self.centroids = {}
    self.past_clusters = {}
    self.clusters = {}

    self.init_clusters()

  def init_clusters(self):
    for key in range(self.k):
      self.past_clusters[key] = []
      self.clusters[key] = []

  def init_centroid(self):
    list_min = self.data.min().to_list()
    list_max = self.data.max().to_list()

    for key in range(self.k):
      centroid_list = []
      for column in range(len(self.data.columns)):
        centroid_list.append(random()*(list_max[column] - list_min[column]) + list_min)
      self.centroids[key] = tuple(centroid_list)

  def calculate_closest_centroid(self, row):
    distances_cluster = []
    for key in self.clusters.keys():
      distances_cluster.append((self.calc_distance(self.clusters[key], row)))

    return distances_cluster.index(min(distances_cluster))

  def main_loop(self):
    while (1):
      self.past_clusters = self.clusters
      for row in range(len(self.data)):
        closest_centroid = self.calculate_closest_centroid(self.data.iloc[row, :])
        self.clusters[closest_centroid].append(self.data.iloc[row, :])


  @staticmethod
  def calc_distance(vec1, vec2):
    return numpy.linalg.norm(vec1-vec2)
    """
    distance = 0
    for i in range(len(vec1)):
      distance += (vec1[i] - vec2[i])**2
    return sqrt(distance)
    """


  def check_variation(self):
    pass



"""
------OPERACIONES------
Saber si un elemento pertenece a un cluster
Saber el centroide del cluster
Saber si no hay variación
"""



"""
k = numero de clusters [CHECK]
inicializas k clusters aleatoriamente (centroides aleatorios) [CHECK]
Calcular distancias a los centroides [CHECK]
metes cada punto al cluster elegido por el centroide más cercano
recalculas el centroide de cada clase
repites while variación != 0
"""

26.004435261124208
